In [19]:
import glob
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import numpy as np
import datetime

def load_data(directory):
    files = glob.glob(f"{directory}/**/*.csv", recursive=True)
    df_list = []
    file_count = 0
    for f in files:
        file_count += 1
        try:
            temp_df = pd.read_csv(f)
            if not temp_df.empty: 
                df_list.append(temp_df)
        except pd.errors.EmptyDataError:
            print(f'File {f} is empty, skipping.')
    print('All files processed.')
    df = pd.concat(df_list, ignore_index=True)
    df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
    return df

def clean_data(df):
    df.replace(['N/A', r"\\t|\\n|\\r", "\t|\n|\r"], [np.nan, "", ""], regex=True, inplace=True)
    df.dropna(inplace=True)
    df['Course'] = df['Course'].str.replace(r'\([A-Z]\)$', '', regex=True)
    df['Avg Grade Expected'] = df['Avg Grade Expected'].str.extract(r'\((.*?)\)', expand=False)
    df['Avg Grade Received'] = df['Avg Grade Received'].str.extract(r'\((.*?)\)', expand=False)
    for col in ['Rcmnd Class', 'Rcmnd Instr']:
        df[col] = df[col].str.rstrip('%').astype('float') / 100.0
    for col in ['Avg Grade Expected', 'Avg Grade Received', 'Study Hrs/wk']:
        df[col] = df[col].astype(float)
    return df

df = load_data('evals')
df = clean_data(df)

def weighted_mean(x, weights):
    return np.average(x, weights=weights)

def term_to_date(term):
    term_mapping = {'FA': '09', 'WI': '01', 'SP': '03', 'S1': '06', 'S2': '08', 'S3': '07'}
    term_season, term_year = term[:2], term[2:]
    if term_year <= str(datetime.datetime.now().year)[-2:]:
        term_year = '20' + term_year
    else:
        term_year = '19' + term_year
    return pd.to_datetime(term_year + term_mapping[term_season] + '01')

df['Term'] = df['Term'].apply(term_to_date)

agg_dict = {
    'Term': 'max',
    'Enroll': 'sum',
    'Evals Made': 'sum',
    'Rcmnd Class': lambda x: weighted_mean(x, weights=df.loc[x.index, 'Evals Made']),
    'Rcmnd Instr': lambda x: weighted_mean(x, weights=df.loc[x.index, 'Evals Made']),
    'Study Hrs/wk': lambda x: weighted_mean(x, weights=df.loc[x.index, 'Evals Made']),
    'Avg Grade Expected': lambda x: weighted_mean(x, weights=df.loc[x.index, 'Evals Made']),
    'Avg Grade Received': lambda x: weighted_mean(x, weights=df.loc[x.index, 'Enroll'])
}

df = df.groupby(['Instructor', 'Course']).agg(agg_dict).reset_index()

df['Term'] = df['Term'].apply(lambda x: {1: 'WI', 3: 'SP', 6: 'S1', 7: 'S3', 8: 'S2', 9: 'FA'}[x.month] + str(x.year)[-2:])
df['Rcmnd Class'] = df['Rcmnd Class'].apply(lambda x: '{:.1%}'.format(x))
df['Rcmnd Instr'] = df['Rcmnd Instr'].apply(lambda x: '{:.1%}'.format(x))
df['Study Hrs/wk'] = df['Study Hrs/wk'].round(2)
df['Avg Grade Expected'] = df['Avg Grade Expected'].round(2)
df['Avg Grade Received'] = df['Avg Grade Received'].round(2)
df['Department'] = df['Course'].str.split().str[0]
df['CourseNumber'] = df['Course'].str.split().str[1]

column_select_primary = widgets.Dropdown(
    options=df.columns.tolist(),
    value='Instructor',
    description='Primary Sort:',
)
column_select_secondary = widgets.Dropdown(
    options=df.columns.tolist(),
    value='Course',
    description='Secondary Sort:',
)
sort_order_primary = widgets.Dropdown(
    options=['Ascending', 'Descending'],
    value='Ascending',
    description='Primary Order:',
)
sort_order_secondary = widgets.Dropdown(
    options=['Ascending', 'Descending'],
    value='Ascending',
    description='Secondary Order:',
)

department_select = widgets.Dropdown(
    options=['All'] + df['Department'].unique().tolist(),
    value='All',
    description='Department:',
)

course_select = widgets.Dropdown(
    options=['All'] + df['CourseNumber'].unique().tolist(),
    value='All',
    description='Course Number:',
)

def on_button_clicked(b):
    clear_output(wait=True)
    
    primary_column = column_select_primary.value
    secondary_column = column_select_secondary.value
    primary_order = True if sort_order_primary.value == 'Ascending' else False
    secondary_order = True if sort_order_secondary.value == 'Ascending' else False

    selected_department = department_select.value
    selected_course = course_select.value

    temp_df = df.copy()
    if selected_department != 'All':
        temp_df = temp_df[temp_df['Department'] == selected_department]
    if selected_course != 'All':
        temp_df = temp_df[temp_df['CourseNumber'] == selected_course]

    display(temp_df.sort_values(by=[primary_column, secondary_column], ascending=[primary_order, secondary_order]).head(50))

    display(department_select)
    display(course_select)
    display(column_select_primary)
    display(sort_order_primary)
    display(column_select_secondary)
    display(sort_order_secondary)
    display(sort_button)

display(department_select)
display(course_select)
display(column_select_primary)
display(sort_order_primary)
display(column_select_secondary)
display(sort_order_secondary)
display(sort_button)


File evals\cse\cse-180r.csv is empty, skipping.
File evals\cse\cse-195.csv is empty, skipping.
File evals\cse\cse-197.csv is empty, skipping.
File evals\cse\cse-197c.csv is empty, skipping.
File evals\cse\cse-199.csv is empty, skipping.
File evals\cse\cse-199h.csv is empty, skipping.
File evals\cse\cse-99.csv is empty, skipping.
All files processed.


Dropdown(description='Department:', options=('All', 'CHEM', 'MATH', 'ESYS', 'PHYS', 'MUS', 'BENG', 'ECE', 'MAE…

Dropdown(description='Course Number:', options=('All', '167', '11', '181B', '183', '20E', '102', '130A', '5', …

Dropdown(description='Primary Sort:', options=('Instructor', 'Course', 'Term', 'Enroll', 'Evals Made', 'Rcmnd …

Dropdown(description='Primary Order:', options=('Ascending', 'Descending'), value='Ascending')

Dropdown(description='Secondary Sort:', index=1, options=('Instructor', 'Course', 'Term', 'Enroll', 'Evals Mad…

Dropdown(description='Secondary Order:', options=('Ascending', 'Descending'), value='Ascending')

Button(description='Sort Data', style=ButtonStyle())